In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 글꼴깨짐 방지
import matplotlib
import matplotlib.font_manager as fm

import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

fm.get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

In [2]:
df = pd.read_csv('C:/Users/win10/Downloads/235745_parking_data/train.csv')
df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [3]:
def preprocessing(df):
    # 오류 단지코드가 존재하는 행들을  사전에 제거
    df_error =  ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
    #df_error =  ['C2335', 'C1327']
    df = df[~df['단지코드'].isin(df_error)].reset_index(drop=True)
    df.rename(columns = {'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철','도보 10분거리 내 버스정류장 수':'버스'},inplace=True)
    df.drop(columns=['임대보증금','임대료','자격유형','임대건물구분'],axis = 1,inplace=True)
    지역_비율 = (df.groupby(['지역'])['총세대수'].count())/(df.groupby(['지역'])['총세대수'].count().sum())*100
    지역_비율=지역_비율.reset_index(name='지역_비율')
    공급유형_비율 = (df.groupby(['공급유형'])['총세대수'].count())/(df.groupby(['공급유형'])['총세대수'].count().sum())*100
    공급유형_비율=공급유형_비율.reset_index(name='공급유형_비율')
    df = pd.merge(df,지역_비율, on='지역')
    df = pd.merge(df,공급유형_비율, on='공급유형')
    df.drop(columns=['지역','공급유형','단지코드'],axis = 1,inplace=True)
    df=df.dropna(axis=0)
    df = df[['총세대수', '전용면적', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수', '공급유형_비율',
       '지역_비율', '등록차량수']]
    return df

In [4]:
df = preprocessing(df)

In [5]:
!pip install XGBoost

Defaulting to user installation because normal site-packages is not writeable


In [6]:
#Splitting the data into train and test split
from sklearn.model_selection import train_test_split
X = df.iloc[:, 1:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

from time import time

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [8]:
regressors = [
    KNeighborsRegressor(),
    GradientBoostingRegressor(),
    KNeighborsRegressor(),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
    XGBRegressor(),
    LinearRegression(),
    Lasso(),
    Ridge()
]

In [9]:
head = 10
for model in regressors[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))
    print()

KNeighborsRegressor()
	Training time: 0.002s
	Prediction time: 0.003s
	Explained variance: 0.8260783220523491
	Mean absolute error: 108.38919925512104
	R2 score: 0.8260188657053464

GradientBoostingRegressor()
	Training time: 0.135s
	Prediction time: 0.002s
	Explained variance: 0.924364697996993
	Mean absolute error: 80.24586319771744
	R2 score: 0.9243483477911816

KNeighborsRegressor()
	Training time: 0.003s
	Prediction time: 0.002s
	Explained variance: 0.8260783220523491
	Mean absolute error: 108.38919925512104
	R2 score: 0.8260188657053464

ExtraTreesRegressor()
	Training time: 0.158s
	Prediction time: 0.010s
	Explained variance: 0.9975077157804304
	Mean absolute error: 7.459702048417131
	R2 score: 0.9975029767386651

RandomForestRegressor()
	Training time: 0.316s
	Prediction time: 0.010s
	Explained variance: 0.9858710223818705
	Mean absolute error: 22.090093109869642
	R2 score: 0.9858522085341397

DecisionTreeRegressor()
	Training time: 0.005s
	Prediction time: 0.000s
	Explained va

In [10]:
parameters = {'n_estimators':[100,200],
              'learning_rate' : [0.01,0.1,0.15,0.3,0.5],
              'max_depth':[3,6,9],#트리의 최대 깊이
              'gamma' : [0],#분할을 수행하는데 필요한 최소 손실 감소를 지정, loss function에 따라 조정해야함
              'colsample_bytree':[0.9],#각 트리마다의 feature 샘플링 비율
              'eval_metric': ['mae'],
              'scale_pos_weight':[0.5],
              'subsample': [0.5],#각 트리마다의 관측 데이터 샘플링 비율, 값을 적게 주면 오버피팅, 더 작게 주면 언더 피팅
              'min_child_weight': [1,5,10,20,22],
              'seed': [1337],
              'reg_alpha':[0.001,0.01,0.1],
              'reg_lambda':[0.001,0.01,0.1]
             }

In [11]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(XGBRegressor(),parameters,verbose=10)
model = grid.fit(X,y)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

Fitting 5 folds for each of 1350 candidates, totalling 6750 fits
[CV 1/5; 1/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.001, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 1/5; 1/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.001, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.034 total time=   0.0s
[CV 2/5; 1/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.001, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 2/5; 1/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.001, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.

[CV 4/5; 4/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 4/5; 4/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.593 total time=   0.0s
[CV 5/5; 4/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 5/5; 4/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.220 total time=   0.0s
[CV 1/5; 5/1350] START colsample_bytree=0.9, e

[CV 2/5; 8/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.1, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 2/5; 8/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.1, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.115 total time=   0.0s
[CV 3/5; 8/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.1, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 3/5; 8/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, reg_alpha=0.1, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.142 total time=   0.0s
[CV 4/5; 8/1350] START colsample_bytree=0.9, eval_metr

[CV 5/5; 11/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.001, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 5/5; 11/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.001, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.547 total time=   0.0s
[CV 1/5; 12/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.001, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 1/5; 12/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.001, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.452 total time=   0.0s
[CV 2/5; 12/1350] START colsample_bytree=0.9

[CV 3/5; 15/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.01, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 3/5; 15/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.01, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.647 total time=   0.0s
[CV 4/5; 15/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.01, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 4/5; 15/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, reg_alpha=0.01, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.756 total time=   0.0s
[CV 5/5; 15/1350] START colsample_bytree=0.9, eval

...

[CV 4/5; 1335/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.001, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.722 total time=   0.0s
[CV 5/5; 1335/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.001, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 5/5; 1335/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.001, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.532 total time=   0.0s
[CV 1/5; 1336/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 1/5; 1336/1350] END colsample_byt

[CV 2/5; 1339/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.1, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.808 total time=   0.0s
[CV 3/5; 1339/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.1, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 3/5; 1339/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.1, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.537 total time=   0.0s
[CV 4/5; 1339/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=100, reg_alpha=0.1, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 4/5; 1339/1350] END colsample_bytr

[CV 5/5; 1342/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.001, reg_lambda=0.001, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.599 total time=   0.0s
[CV 1/5; 1343/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.001, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 1/5; 1343/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.001, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.290 total time=   0.0s
[CV 2/5; 1343/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.001, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 2/5; 1343/1350] END colsample

[CV 3/5; 1346/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.01, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.549 total time=   0.0s
[CV 4/5; 1346/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.01, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 4/5; 1346/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.01, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.716 total time=   0.0s
[CV 5/5; 1346/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.01, reg_lambda=0.01, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 5/5; 1346/1350] END colsample_bytr

[CV 1/5; 1350/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.284 total time=   0.0s
[CV 2/5; 1350/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 2/5; 1350/1350] END colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5;, score=0.837 total time=   0.0s
[CV 3/5; 1350/1350] START colsample_bytree=0.9, eval_metric=mae, gamma=0, learning_rate=0.5, max_depth=9, min_child_weight=22, n_estimators=200, reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=0.5, seed=1337, subsample=0.5
[CV 3/5; 1350/1350] END colsample_bytree=0.9, 

In [12]:
model

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    predictor=None, random_state=None, ...),
             param_grid={'colsample_bytree': [0.9], 'eval_metric': ['mae'],
                         'gamma': [0],
                         'learning_rate': [0.01, 0.1, 0.15, 0.3, 0.5],
                         'max_depth': [3, 6

In [13]:
scores_df = pd.DataFrame(grid.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_eval_metric,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,...,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.050877,0.003676,0.002405,0.000490,0.9,mae,0,0.01,3,1,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.033876,0.115297,0.140031,0.592579,0.220208,0.220398,0.195353,1311
1,0.046582,0.001789,0.002406,0.000491,0.9,mae,0,0.01,3,1,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.033805,0.115144,0.142482,0.593012,0.220228,0.220934,0.195349,1306
2,0.046582,0.001751,0.002443,0.000539,0.9,mae,0,0.01,3,1,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.038009,0.110274,0.142310,0.592095,0.215762,0.219690,0.194766,1312
3,0.047321,0.001866,0.002406,0.000491,0.9,mae,0,0.01,3,1,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.033876,0.115297,0.140030,0.592579,0.220292,0.220415,0.195353,1309
4,0.048424,0.001918,0.002607,0.000491,0.9,mae,0,0.01,3,1,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.033805,0.115144,0.142482,0.593011,0.220228,0.220934,0.195349,1307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,0.142085,0.001399,0.002405,0.000491,0.9,mae,0,0.5,9,22,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.295309,0.803953,0.548972,0.716441,0.546035,0.582142,0.174306,1039
1346,0.141787,0.002488,0.002606,0.000491,0.9,mae,0,0.5,9,22,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.299657,0.834196,0.452661,0.719680,0.521355,0.565510,0.190433,1126
1347,0.142820,0.001923,0.002696,0.000587,0.9,mae,0,0.5,9,22,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.292568,0.806208,0.538935,0.685082,0.620415,0.588642,0.171916,987
1348,0.140823,0.002092,0.002906,0.000201,0.9,mae,0,0.5,9,22,...,0.5,"{'colsample_bytree': 0.9, 'eval_metric': 'mae'...",0.279855,0.798219,0.531296,0.718115,0.579854,0.581468,0.178406,1046


In [14]:
grid.best_score_

0.7011757282943146

In [15]:
import joblib

joblib.dump(model, 'XGBRegressor.pkl')

['XGBRegressor.pkl']

In [16]:
XGBRegressor = joblib.load('XGBRegressor.pkl')
pred = XGBRegressor.predict(X)
pred

array([366.1211 , 460.2261 , 442.50336, ..., 691.625  , 737.0599 ,
       813.6642 ], dtype=float32)